<a href="https://colab.research.google.com/github/anqitu/NTUOSS-AlgorithmicTradingWorkshop/blob/master/completed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Task 1 - Set Up

#### 1.1 Import Necessary Libraries

In [0]:
# TASK 1.1: Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

#### 1.2 Import Financial Data

In [0]:
# TASK 1.2: Import Apple's stock market price from 2006 to 2019
import pandas_datareader as pdr
import datetime 
aapl = pdr.get_data_yahoo('AAPL', 
                          start=datetime.datetime(2006, 10, 1), 
                          end=datetime.datetime(2019, 1, 1))


## Task 2 - Work with Pandas Dataframe

#### 2.1 Check Data

In [0]:
# Task 2.1.1 Check the structure of a Pandas Dataframe
print(aapl)

In [0]:
# Task 2.1.2 Use the describe() function to get some useful summary statistics about your data
print(aapl.describe(include = 'all'))

In [0]:
# Task 2.1.3 Check the general information of the dataframe
print(aapl.info())

In [0]:
# Task 2.1.4 Check the shape, columns and index of a Pandas Dataframe
print(aapl.shape)
print(aapl.columns)
print(aapl.index)

In [0]:
# Task 2.1.5 Check first few rows of dataframe
print(aapl.head())

In [0]:
# Task 2.1.6 Check last 10 rows of dataframe
print(aapl.tail(10))

In [0]:
# Task 2.1.7 Check the existence of null values
print(aapl.isnull().sum())

#### 2.2 Clean Data

In [0]:
# Task 2.2.1 Drop a column
aapl = aapl.drop(columns = ['High', 'Low'])

In [0]:
# Check the column has been dropped
print(aapl.head())

In [0]:
# Task 2.2.2 Keep wanted columns
aapl = aapl[['Close', 'Adj Close']]

In [0]:
print(aapl.head())

## Task 3 - Visualize Data (Exploratory Data Analysis)

In [0]:
# Task 3.1 Plot the closing price
aapl['Close'].plot()

In [0]:
# Task 3.2 Plot the closing price with some nice adjustments
aapl['Close'].plot(figsize = (15,5), linewidth = 1, legend = True)

In [0]:
# Task 3.2 Plot the closing and adjusted closing prices
aapl['Close'].plot(figsize = (15,5), linewidth = 1, legend = True)
aapl['Adj Close'].plot(figsize = (15,5), linewidth = 1, legend = True)
plt.legend(['Close', 'Adj Close'])
plt.title('Apple Stock Price')
plt.show()

## Task 4 - Build Your Trading Strategy

In [0]:
# Task 4.1 Initialize the short and long windows
short_window = 50
long_window = 100

In [0]:
# Task 4.2 Create moving averages over the short window
aapl['short_ma'] = aapl['Close'].rolling(window = short_window, min_periods = 1).mean()
aapl['long_ma'] = aapl['Close'].rolling(window = long_window, min_periods = 1).mean()

In [0]:
# Check the dataframe
aapl

In [0]:
# Task 4.3 Initilize as 0
aapl['higher_short_ma'] = 0

In [0]:
# Task 4.4 Set 1 for rows where short_ma is higher
aapl['higher_short_ma'][short_window:] = np.where(aapl['short_ma'][short_window:] > aapl['long_ma'][short_window:], 1, 0)   

In [0]:
# Task 4.5 Print rows where higher_short_ma is of value 1
aapl[aapl['higher_short_ma'] == 1]

In [0]:
# Task 4.6 Generate trading signals
aapl['Signal'] = aapl['higher_short_ma'].diff()

In [0]:
# Check days when there is a trading signal
aapl[aapl['Signal'] != 0]

In [0]:
# Check the data in December of 2006 when buying happens
aapl['2006-12':].head(15)

In [0]:
# Check the data in February of 2008 when selling happens
aapl['2008-02':].head(15)

In [0]:
## Task 4.6 plot the short and long moving averages, together with the buy and sell signals

# Initialize the plot figure
fig = plt.figure(figsize = (15,5))

# Add a subplot and label for y-axis
ax1 = fig.add_subplot(111,  ylabel='Price')

# Plot the closing price
aapl['Close'].plot(ax = ax1, color = 'r', linewidth = 1, legend = True)

# Plot the short and long moving averages
aapl[['short_ma', 'long_ma']].plot(ax = ax1, linewidth = 1, legend = True)

# Plot the buy signals
ax1.plot(aapl[aapl['Signal'] == 1].index, 
         aapl[aapl['Signal'] == 1]['short_ma'], 
         '^', markersize = 10, color = 'r', alpha = 0.6)
         
# Plot the sell signals
ax1.plot(aapl[aapl['Signal'] == -1].index, 
         aapl[aapl['Signal'] == -1]['short_ma'],
         'v', markersize = 10, color = 'g', alpha = 0.6)
         
# Show the plot
plt.show()

## Task 5 - Test Your Strategy

In [0]:
# Task 5.1 Imagine you have 1 million
initial_capital= 1000000

In [0]:
# Task 5.2 Buy or Sell a 5k shares on the days that the signal is 1 or -1
aapl['Order'] = 5000 * aapl['Signal']

In [0]:
# Task 5.3 Calculate the transaction price paying (buy stocks) or receiving (sell stocks)
aapl['Transaction'] = aapl['Order'].multiply(aapl['Adj Close'], axis=0)

In [0]:
# Check data
aapl[aapl['Transaction'] != 0]

In [0]:
# Task 5.4 Calculate the number in shares owned 
aapl['Shares'] = aapl['Order'].cumsum()

In [0]:
# Task 5.5 Calculate the value of shares owned 
aapl['Holdings'] = aapl['Shares'].multiply(aapl['Adj Close'], axis=0)

In [0]:
# Task 5.6 Calculate the value of cash owned 
aapl['Cash'] = initial_capital - (aapl['Transaction']).cumsum()   

In [0]:
# Check the data in December of 2006 when buying happens
aapl['2006-12':].head(15)

In [0]:
# Check the data in February of 2008 when selling happens
aapl['2008-02':].head(15)

In [0]:
# Task 5.7 Calculate the total value of your portfolio
aapl['Total'] = aapl['Cash'] + aapl['Holdings']


In [0]:
# Check the data in December of 2006 when buying happens
aapl['2006-12':].head(15)

In [0]:
# Task 5.8 Visualize the portfolio value or over the years

# Create a figure
fig = plt.figure(figsize = (15,5))

ax1 = fig.add_subplot(111, ylabel='Portfolio value')

# Plot the equity curve
aapl['Total'].plot(ax = ax1, linewidth = 1)

ax1.plot(aapl[aapl['Signal'] == 1].index, 
         aapl[aapl['Signal'] == 1.0]['Total'],
         '^', markersize = 10, color = 'r', alpha = 0.6)
ax1.plot(aapl[aapl['Signal'] == -1].index, 
         aapl[aapl['Signal'] == -1.0]['Total'],
         'v', markersize = 10, color = 'g', alpha = 0.6)

# Show the plot
plt.show()

In [0]:
# Task 5.9 Visualize the portfolio value or over the years

# Resample `aapl` to 12 months, take last observation as value 
yearly = aapl['Total'].resample('Y', convention='end').apply(lambda x: x[-1])
print(yearly)

In [0]:
# Task 5.10 Calculate the yearly return
yearly.pct_change()

In [0]:
# Task 5.11 Plot the yearly return
yearly.pct_change().plot(figsize = (15,5))

In [0]:
# aapl['Total'].plot(figsize = (15,5), linewidth = 1, legend = True)
# googl['Total'].plot(figsize = (15,5), linewidth = 1, legend = True)
# ibm['Total'].plot(figsize = (15,5), linewidth = 1, legend = True)
# amzn['Total'].plot(figsize = (15,5), linewidth = 1, legend = True)
# plt.legend(['AAPL', 'GOOGL', 'IBM', 'AMZN'])
# plt.title('Portfolio Value')
# plt.show()